In [134]:
import pandas as pd
import numpy as np
import os
import datetime as dt
import pytz
import gensim
from nltk.tokenize import TweetTokenizer
from keras.preprocessing.text import Tokenizer
import gc
import string
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
from collections import Counter
import pickle
pd.options.display.max_columns=None

In [2]:
all_files=os.listdir()
mentions=[_ for _ in all_files if _.__contains__("Mentions")]

In [3]:
len(mentions)

44

In [16]:
df=pd.DataFrame()
for filename in mentions:
    try:
        data=pd.read_csv(filename,skiprows=5,parse_dates=[3],usecols=['Source','Link','Time(ET)','LocalTime','Authority','Gender','Country','Location','Assigned','Sentiment','Alexa Rank','Alexa Reach','Title','Snippet','Content','Unique ID','Language','Followers','Following','Summary','Viewed','Commented','Favourited','Rated'])
        data['showname']=filename.split(".")[0]
        df=pd.concat([df,data],axis=0)
    except e:
        print("filename {} with exception {}".format(filename,e))
        continue
 

S:\Anaconda\envs\py36\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (67) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df.shape

(827641, 25)

In [11]:
df.head()

,Source,Link,Time(ET),LocalTime,Authority,Gender,Country,Location,Assigned,Sentiment,Alexa Rank,Alexa Reach,Title,Snippet,Content,Unique ID,Language,Followers,Following,Summary,Viewed,Commented,Favourited,Rated,showname
0,TWITTER,http://twitter.com/ASHWANI11345063/statuses/10...,Oct 25 2018 23:29:56 EDT,2018-10-25 23:29:56,0,Male,NaN,NaN,none+,None,0,0,@ASHWANI11345063,@AndTVOfficial YE JO PERFECT PATI KA SHOW HAI ...,@AndTVOfficial YE JO PERFECT PATI KA SHOW HAI ...,23723083,TL,1,8,NaN,0,0,0,0,andtv-show-perfect-pati-Mentions
1,INSTAGRAM,https://www.instagram.com/p/BpXjXNVC03A/,Oct 25 2018 15:34:56 EDT,2018-10-25 15:34:56,0,NaN,BG,Sofia;-42;-Bulgaria,none+,Positive,0,0,dabh_big_fan_bulgaria,"Ayush Anand in ,,Perfect Pati"" 25/10/18 Episod...","Ayush Anand in ,,Perfect Pati"" 25/10/18 Episod...",23709925,EN,0,0,NaN,0,0,0,0,andtv-show-perfect-pati-Mentions
2,INSTAGRAM,https://www.instagram.com/p/BpXjSZsiXa5/,Oct 25 2018 15:34:17 EDT,2018-10-25 15:34:17,0,NaN,BG,Sofia;-42;-Bulgaria,none+,Positive,0,0,dabh_big_fan_bulgaria,"Ayush Anand in ,,Perfect Pati"" 25/10/18 Episod...","Ayush Anand in ,,Perfect Pati"" 25/10/18 Episod...",23709924,EN,0,0,NaN,0,0,0,0,andtv-show-perfect-pati-Mentions
3,INSTAGRAM,https://www.instagram.com/p/BpXJdcBFcCf/,Oct 25 2018 11:53:50 EDT,2018-10-25 11:53:50,0,NaN,ZZ,NaN,none+,None,0,0,perfectpati__,#perfectpati #episode 3 When #pushkar #comes #...,#perfectpati #episode 3 When #pushkar #comes #...,23693611,English,0,0,NaN,0,0,0,0,andtv-show-perfect-pati-Mentions
4,INSTAGRAM,https://www.instagram.com/p/BpWlo3JBniN/,Oct 25 2018 06:38:08 EDT,2018-10-25 06:38:08,0,NaN,ZZ,NaN,none+,Positive,0,0,jayaprada_unofficial,Everyone looking amazing #onlocation #shooting...,Everyone looking amazing #onlocation #shooting...,23667295,EN,0,0,NaN,0,0,0,0,andtv-show-perfect-pati-Mentions


In [27]:
# a="zeetv-shows-tujhse-hai-rabta -Mentions.csv"
# a.split(".")[0]

'zeetv-shows-tujhse-hai-rabta -Mentions'

In [4]:
# data=pd.read_csv("zeetv-shows-tujhse-hai-rabta -Mentions.csv",parse_dates=[3],skiprows=5,usecols=['Source','Link','Time(ET)','LocalTime','Authority','Gender','Country','Location','Assigned','Sentiment','Alexa Rank','Alexa Reach','Title','Snippet','Content','Unique ID','Language','Followers','Following','Summary','Viewed','Commented','Favourited','Rated'])


In [ ]:
# ,date_parser=lambda x:dt.datetime.strptime(x,"%b %d %Y %H:%M:%S %z")

In [13]:
showrename={
    'zeetv---show---kaleerein 2-Mentions':'zeetv---show---kaleerein -Mentions',
    'zee-tv---show---juzzbaaat 2-Mentions':'zee-tv---show---juzzbaaat-Mentions',
    'sony-tv-show-indian-idol 1-Mentions':'sony-tv-show-indian-idol -Mentions',
    'sony-tv-show-indian-idol 2-Mentions':'sony-tv-show-indian-idol -Mentions',
    'sony-tv-show-indian-idol 3-Mentions':'sony-tv-show-indian-idol -Mentions',
    'sony-tv-show-dus-ka-dum 1-Mentions':'sony-tv-show-dus-ka-dum -Mentions',
    'sony-tv-show-dus-ka-dum 2-Mentions':'sony-tv-show-dus-ka-dum -Mentions',
    'sony-tv-show-dus-ka-dum 3-Mentions':'sony-tv-show-dus-ka-dum -Mentions',
    'colors-tv-show-naagin3 1-Mentions':'colors-tv-show-naagin3 -Mentions',
    'colors-tv-show-naagin3 2-Mentions':'colors-tv-show-naagin3 -Mentions',
    'colors-tv-show-naagin3 3-Mentions':'colors-tv-show-naagin3 -Mentions'
}

In [17]:
df['new_name']=df['showname']
for x,y in showrename.items():
    df.loc[df.showname==x,'new_name']=y

In [21]:
df[df.Source=='TWITTER'].groupby(['new_name'])['showname'].count()

new_name
andtv-show-perfect-pati-Mentions                               282
andtvofficial-show-bitti-businesswali-Mentions               43447
andtvofficial-show-high-fever-Mentions                        1693
andtvofficial-show-laal-ishq-Mentions                          115
andtvofficial-show-love-me-india-Mentions                    18649
andtvofficial-show-mitegi-lakshmanrekha-Mentions               672
andtvofficial-show-vikram-betaal-Mentions                     2444
colors-tv---show--dance-deewane-Mentions                     44154
colors-tv-show-dastan-e-mohabbat-Mentions                     2247
colors-tv-show-naagin3 -Mentions                             66098
colors-tv-show-rising-star-Mentions                           4343
colorstv-show-dev2-Mentions                                   4724
colorstv-show-internet-wala-love-Mentions                     3802
colorstv-show-kasam-Mentions                                 24201
colorstv-show-kaun-hai-Mentions                      

In [22]:
twitter=df[df.Source=='TWITTER']

In [31]:
twitter['original_tweet']=~twitter.Content.str.startswith(('RT','QT'))

S:\Anaconda\envs\py36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [32]:
twitter.head()

,Source,Link,Time(ET),LocalTime,Authority,Gender,Country,Location,Assigned,Sentiment,Alexa Rank,Alexa Reach,Title,Snippet,Content,Unique ID,Language,Followers,Following,Summary,Viewed,Commented,Favourited,Rated,showname,new_name,original_tweet
0,TWITTER,http://twitter.com/ASHWANI11345063/statuses/10...,Oct 25 2018 23:29:56 EDT,2018-10-25 23:29:56,0,Male,NaN,NaN,none+,None,0,0,@ASHWANI11345063,@AndTVOfficial YE JO PERFECT PATI KA SHOW HAI ...,@AndTVOfficial YE JO PERFECT PATI KA SHOW HAI ...,23723083,TL,1,8,NaN,0,0,0,0,andtv-show-perfect-pati-Mentions,andtv-show-perfect-pati-Mentions,True
5,TWITTER,http://twitter.com/P_Madhumita/statuses/105540...,Oct 25 2018 06:22:06 EDT,2018-10-25 06:22:06,5,Male,NaN,NaN,none+,None,0,0,@P_Madhumita,"RT @isalilsand: #AndTV: #Agnifera 0.2, #Vikram...","RT @isalilsand: #AndTV: #Agnifera 0.2, #Vikram...",23665380,ID,264,82,NaN,0,0,0,0,andtv-show-perfect-pati-Mentions,andtv-show-perfect-pati-Mentions,False
7,TWITTER,http://twitter.com/Rohiths2001/statuses/105538...,Oct 25 2018 05:13:10 EDT,2018-10-25 05:13:10,1,NaN,IN,;-;-India,none+,None,0,0,@Rohiths2001,"RT @isalilsand: #AndTV: #Agnifera 0.2, #Vikram...","RT @isalilsand: #AndTV: #Agnifera 0.2, #Vikram...",23660437,ID,16,98,NaN,0,0,0,0,andtv-show-perfect-pati-Mentions,andtv-show-perfect-pati-Mentions,False
8,TWITTER,http://twitter.com/veekay_ness/statuses/105538...,Oct 25 2018 05:02:06 EDT,2018-10-25 05:02:06,3,Male,IN,mumbai;-maharashtra;-India,none+,Positive,0,0,@veekay_ness,RT @GossipsTv: #SuperExclusive #TRP #TRPDay @A...,RT @GossipsTv: #SuperExclusive #TRP #TRPDay @A...,23659646,EN,67,659,NaN,0,0,0,0,andtv-show-perfect-pati-Mentions,andtv-show-perfect-pati-Mentions,False
9,TWITTER,http://twitter.com/GossipsTv/statuses/10553797...,Oct 25 2018 04:45:25 EDT,2018-10-25 04:45:25,7,NaN,IN,mumbai;-maharashtra;-India,none+,Positive,0,0,@GossipsTv,#SuperExclusive #TRP #TRPDay @AndTVOfficial Al...,#SuperExclusive #TRP #TRPDay @AndTVOfficial Al...,23658245,EN,3596,44,NaN,0,0,0,0,andtv-show-perfect-pati-Mentions,andtv-show-perfect-pati-Mentions,True


In [33]:
twitter[twitter.original_tweet].groupby('new_name')['new_name'].count()

new_name
andtv-show-perfect-pati-Mentions                              132
andtvofficial-show-bitti-businesswali-Mentions              17773
andtvofficial-show-high-fever-Mentions                        339
andtvofficial-show-laal-ishq-Mentions                          12
andtvofficial-show-love-me-india-Mentions                    4096
andtvofficial-show-mitegi-lakshmanrekha-Mentions              156
andtvofficial-show-vikram-betaal-Mentions                     517
colors-tv---show--dance-deewane-Mentions                     9194
colors-tv-show-dastan-e-mohabbat-Mentions                     312
colors-tv-show-naagin3 -Mentions                            15729
colors-tv-show-rising-star-Mentions                           406
colorstv-show-dev2-Mentions                                   609
colorstv-show-internet-wala-love-Mentions                    1352
colorstv-show-kasam-Mentions                                 5162
colorstv-show-kaun-hai-Mentions                               201
c

In [34]:
kaliren=twitter[twitter.new_name=='zeetv---show---kaleerein -Mentions']

In [48]:
kaliren_unique=kaliren[kaliren.original_tweet][['Snippet','Unique ID']].reset_index(drop=True)

In [77]:
unique_tweets=kaliren_unique.Snippet.unique()

In [78]:
len(unique_tweets)
string.l

34109

In [83]:
def tokens(tweets):
    tweets=tweets.lower()
    tokens=tokenizer.tokenize(tweets)
    tokens=filter(lambda x:not x.startswith('http'),tokens)
    return list(tokens)

In [84]:
tokenizer=TweetTokenizer()
# tweets_tokens=kaliren_unique.Snippet.str.lower().map(tokens)
tweets_tokens=[tokens(x) for x in unique_tweets ]

In [126]:
gc.collect()

21693

In [105]:
?model.train

In [102]:
tweets_tokens[11]

['zee',
 "tv's",
 'kaleerein',
 'attempts',
 'at',
 'breaking',
 'wedding',
 'conventions',
 '!',
 '-',
 'click',
 'link',
 'for',
 'more',
 'details',
 ':',
 '#desiserials']

In [106]:
model=Word2Vec(tweets_tokens,size=100,min_count=1)
model.train(tweets_tokens,total_examples=len(tweets_tokens),epochs=10)

(5132100, 7271540)

In [99]:
model.wv['समय']

array([ 2.048611  ,  0.72333616,  0.48724484, -0.4500024 ,  0.16357833,
       -0.52212197, -0.12490384,  0.9180243 ,  0.5167163 , -0.5326188 ,
        0.33422235, -0.30013397,  0.69269574,  0.53496855, -0.49670246,
       -1.9816524 , -0.42951277, -0.31081122, -1.4872102 ,  0.92140067,
        0.43821844,  0.40226784, -0.41806713, -0.33117673, -1.7038857 ,
       -1.3324441 , -0.47213537, -0.50765216,  0.4834022 ,  0.47384185,
        0.06393668, -0.19112277, -0.07786715, -0.05377731,  0.3818289 ,
        0.5372364 , -0.263294  , -0.7916144 ,  0.00329095,  1.1029572 ,
        1.1255887 , -0.1141318 ,  0.2280727 , -0.02793447, -0.13944098,
       -0.6023528 , -0.7301585 , -0.26395828,  0.32110256, -0.09615557,
        0.02769934, -0.6816367 ,  0.165897  , -0.21050382, -0.3308086 ,
        0.42708635, -0.81411725,  0.71814483,  1.4309062 ,  1.1524026 ,
        0.09825108,  0.4357484 ,  1.0902952 ,  0.3902689 ,  0.09506059,
        1.1099333 , -0.64247173,  0.46295503,  0.44847092,  0.25

In [107]:
model.most_similar('kaleerein')

S:\Anaconda\envs\py36\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('#kaleereinonzeetv', 0.53196781873703),
 ('exciting', 0.495624303817749),
 ('new', 0.4914831221103668),
 ('amazing', 0.45753681659698486),
 ('fun', 0.4441685676574707),
 ('awesome', 0.4202868640422821),
 ('story', 0.40998464822769165),
 ('tomorrow', 0.4075630307197571),
 ('air', 0.40713128447532654),
 ('here', 0.4048286974430084)]

In [110]:
model.most_similar('wedding')

S:\Anaconda\envs\py36\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('includes', 0.6195054054260254),
 ('shopping', 0.5377169847488403),
 ('destinations', 0.4993015229701996),
 ('handbell', 0.4979173541069031),
 ('hour', 0.4794900119304657),
 ('groom', 0.47148397564888),
 ('mall', 0.44611525535583496),
 ('bridal', 0.44585755467414856),
 ('tiring', 0.4450395107269287),
 ('bride', 0.4449368715286255)]

In [112]:
keyed=model.wv
del model

In [118]:
embeddings=np.zeros(shape=(len(unique_tweets),100))
for i,tweet in enumerate(tweets_tokens):
    sm=0
    for word in tweet:
        try:
            embeddings[i]+=keyed[word]
        except KeyError:
            print(tweet)
    embeddings[i]=embeddings[i]/len(tweet)

In [120]:
embeddings[0]

array([ 0.08311675,  0.00180947,  0.11284693,  0.26363971, -0.20051023,
       -0.06370926, -0.24295395, -0.25054403, -0.32422774, -0.29224951,
        0.27901908, -0.02849683,  0.17164341,  0.03295574,  0.02017123,
       -0.20996328, -0.26678178,  0.12376142, -0.13043866,  0.13306908,
       -0.0465849 ,  0.2439157 , -0.04600697, -0.17936138,  0.02079167,
       -0.26478318,  0.05012254, -0.11957114, -0.03049924, -0.16799068,
       -0.05896449, -0.08186254, -0.01065982, -0.11900656, -0.04999774,
       -0.32816371,  0.12494754, -0.0991888 ,  0.13281398,  0.15308093,
       -0.2709503 ,  0.05591038, -0.02954648, -0.01190412,  0.05372787,
        0.20159995, -0.16913732,  0.02146907,  0.0643775 , -0.03394542,
       -0.1378218 , -0.29003948,  0.23668791,  0.04646603,  0.03406822,
        0.21337168, -0.04866543, -0.15255411,  0.06339086,  0.02689708,
        0.26362369,  0.06767513,  0.20080405, -0.08234051, -0.31500502,
       -0.20921972,  0.03241968, -0.02142066, -0.04093158,  0.02

In [129]:
km=KMeans(n_clusters=8,n_jobs=-1)
km.fit(embeddings)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=8, n_init=10, n_jobs=-1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [131]:
Counter(km.labels_)

Counter({0: 6944,
         1: 1264,
         2: 6700,
         3: 4489,
         4: 1822,
         5: 5239,
         6: 6419,
         7: 1232})

In [ ]:
km.

In [132]:
def fit_clusters(clf,filename):
    clf.fit(embeddings)
    df_tweets=pd.DataFrame({'tweets':unique_tweets,'clusters':clf.labels_})
    df_tweets.to_csv(filename)

In [133]:
fit_clusters(KMeans(n_clusters=8,n_jobs=-1),"cluster_8.csv")

In [137]:
with open("embeddings",'wb') as fw:
    pickle.dump(embeddings,fw)